### Some notes:

- Fixed size
- Contiguous block of memory
- O(1) to store and access an element
- Strings, stacks, queues, and hash tables use arrays for the implementation

### Limitation

- Looking up an element by value requires an entire traversal of the array (unless it's sorted)
- Deleting an element makes all subsequent elements have to be shifted left by one => O(n) time operation
- Similar to inserting
- Fixed size => should be careful of off-by-one errors
- - Python does not have native support for arrays => use `list`, which dynamically resizes under the hood

## Problem 1: Get product of all other elements

Given an array of integers, return a new array such that each element at index i of the new array is the product of all the numbers in the original array except the one at i.   
For example: 
- If our input was \[ 1, 2, 3, 4, 5\], the expected output would be \[120, 60, 40, 30, 24\].
- If our input was \[3, 2, 1\], the expected output would be \[2, 3, 6\].

Follow-up: What if you can't use division?

#### Main Topic: Prefix and Suffix Product Arrays (Product of Array Except Self)

#### The Idea to Solve the Problem
We want each result[i] to be the product of all numbers in the array except nums[i] — but without using division.

#### Key idea:
- Use prefix products (product of all elements to the left).
- Use suffix products (product of all elements to the right).   
  => Use 2 loops, not 1
- Multiply them to get the final result.
- This gives O(n) time and O(1) extra space (if we reuse the result array).

#### Pseudo Code

In [ ]:
function product_except_self(nums):
    1. Initialize the result array of size n with 1s.
    2. Compute prefix products:   
         - prefix = 1   
         - for i in range(n):   
              result[i] = prefix   
              prefix *= nums[i]   
    3. Compute suffix products:   
         - suffix = 1   
         - for j in range(n-1, -1, -1):    
              result[j] *= suffix   
              suffix *= nums[j]   
    4. Return result   

**In the first loop**:
- The first (index 0) element will be 1 (first value of prefix)
- The second (index 1) element will be the product of 1 and the first element (1 * nums\[0\])
- The third (index 2) element will be the product of 1 * nums\[0\] * nums\[1\]
- ...
- The nearly last element (index n-2) will be the product of 1 * nums\[0\] * nums\[1\] * ... * nums\[n-3\]
- The last element (index n-1) will be the product of 1 * nums\[0\] * nums\[1\] * ... * nums\[n-2\]
- You can see that the last element has the value we need

**In the second loop**:
- Note that j starts with n-1, not 0 as the first loop, so result\[j\] in the first iteration is result\[n-1\]
- The last element has the value we need (mentioned above), so it multiplies by 1
- Gradually, the suffix will be the product of 1 * nums\[n-1\] * numns\[n-2\]..... 
- The nearly last element (index n-2) has the value: product of 1 * nums\[0\] * nums\[1\] * ... * nums\[n-3\], now it multiplies by result\[n-1\] * 1
- ...
- Until the end of the loop, it will be done.
- The second loop will loop from n-1 to 0. In the last iteration of this loop, the suffix now is the product of nums\[n-1\] *  nums \[n-2\] * ... * nums\[1\] (note that nums\[1\], not nums\[0\])

#### Code - Python

In [17]:
def product_except_self(nums):
    n = len(nums)
    result = [1] * n

    prefix = 1
    for i in range(n):
        result[i] = prefix
        prefix *= nums[i]
    suffix = 1
    for j in range(n-1, -1, -1):
        result[j] *= suffix
        suffix *= nums[j]
    return result

In [18]:
# Example Usage
print(product_except_self([1, 2, 3, 4, 5])) 
print(product_except_self([4, 7, 3]))       

[120, 60, 40, 30, 24]
[21, 12, 28]


#### Code - Golang

In [19]:
package main

import "fmt"

func productExceptSelf(nums []int) []int {
    n := len(nums)
    result := make([]int, n)
    
    prefix := 1
    for i:=0; i<n; i++ {
        result[i] = prefix
        prefix *= nums[i]
    }

    suffix := 1
    for j:=n-1; j>=0; j-- {
        result[j] *= suffix
        suffix *= nums[j]
    }
    
    return result
}

func main() {
    test1 := []int{1,2,3,4,5}
    test2 := []int{4, 7, 3}
    fmt.Println(productExceptSelf(test1))
    fmt.Println(productExceptSelf(test2))
}

[120 60 40 30 24]
[21 12 28]


## Problem 2: Locate smallest window to be sorted

### Problem 3: Calculate maximum subarray sum

## Problem 4: Find number of smaller elements to the right